In [50]:
import pandas as pd

first_buildings = pd.read_csv("sql/first_buildings.csv")
print(first_buildings)

       gametime         platformgameid  teamid buildingtype
0        710694  ESPORTSTMNT01:1431635     100       turret
1       1863543  ESPORTSTMNT03:2073826     100    inhibitor
2        655449  ESPORTSTMNT02:1331126     200       turret
3       1458069  ESPORTSTMNT03:2542608     200    inhibitor
4       1018478  ESPORTSTMNT05:2942358     100       turret
...         ...                    ...     ...          ...
74992   1440023  ESPORTSTMNT05:1870222     100    inhibitor
74993   2349105  ESPORTSTMNT01:2693353     200    inhibitor
74994    598202  ESPORTSTMNT06:1720693     200       turret
74995    988622  ESPORTSTMNT01:3408793     200       turret
74996   1606464  ESPORTSTMNT03:3185926     100    inhibitor

[74997 rows x 4 columns]


In [51]:
first_inhib = first_buildings[first_buildings["buildingtype"] == "inhibitor"]
first_inhib = first_inhib.sort_values(by=['platformgameid', 'gametime'])
first_inhib["killerteamid_inhib"] = first_inhib["teamid"].apply(lambda x: 200 if x == 100 else 100)
first_inhib.drop_duplicates(subset=['platformgameid'], keep='first', inplace=True)
first_inhib.drop(columns=["buildingtype", "teamid"], inplace=True)
first_inhib.rename(columns={"gametime": "gametime_inhib"}, inplace=True)
display(first_inhib)

,gametime_inhib,platformgameid,killerteamid_inhib
70824,1708748,ESPORTSTMNT01:1110148,100
41143,1602302,ESPORTSTMNT01:1110162,100
62898,1546286,ESPORTSTMNT01:1110177,100
3623,1628728,ESPORTSTMNT01:1110187,100
50722,1694081,ESPORTSTMNT01:1110198,100
...,...,...,...
2262,1700113,LPL_B:330027,100
35688,1658796,LPL_B:330032,200
11354,1946801,LPL_B:330033,200
52312,1447915,LPL_B:330039,200


In [52]:
#first tower
first_tower = first_buildings[first_buildings["buildingtype"] == "turret"]
first_tower = first_tower.sort_values(by=['platformgameid', 'gametime'])
first_tower["killerteamid_tower"] = first_tower["teamid"].apply(lambda x: 200 if x == 100 else 100)
first_tower.drop_duplicates(subset=['platformgameid'], keep='first', inplace=True)
first_tower.drop(columns=["buildingtype", "teamid"], inplace=True)
first_tower.rename(columns={"gametime": "gametime_tower"}, inplace=True)

In [53]:
buildings_joined = first_inhib.merge(first_tower, on="platformgameid")

In [54]:
first_kill = pd.read_csv("sql/first_blood.csv")
first_kill = first_kill.sort_values(by=['platformgameid', 'gametime'])
first_kill.drop_duplicates(subset=['platformgameid'], keep='first', inplace=True)
display(first_kill)
first_kill.rename(columns={
    "gametime": "gametime_kill",
    "killerteamid": "killerteamid_kill",
}, inplace=True)

,gametime,platformgameid,killerteamid
4535,245612,ESPORTSTMNT01:1110148,100
9142,192537,ESPORTSTMNT01:1110162,100
21281,207700,ESPORTSTMNT01:1110177,100
35484,359245,ESPORTSTMNT01:1110187,200
11323,148483,ESPORTSTMNT01:1110198,100
...,...,...,...
21378,294277,LPL_B:330027,100
13430,227140,LPL_B:330032,100
44132,505054,LPL_B:330033,100
17859,302191,LPL_B:330039,100


In [55]:
# add start times
df = buildings_joined.merge(first_kill, on="platformgameid")
event_start_times = pd.read_csv("sql/event_start_times.csv")
df = df.merge(event_start_times, on="platformgameid")

In [57]:
# add team games 
team_games = pd.read_csv("sql/team_games.csv")
team_games.sort_values(by=['platformgameid'], inplace=True)

with_teams = df.merge(team_games, on="platformgameid")

blue_side = with_teams[with_teams["side"] == 100]
blue_side.rename(columns={
    "teamid": "blue_teamid",
}, inplace=True)
blue_side.drop(columns=["side"], inplace=True)
red_side = with_teams[with_teams["side"] == 200]
red_side.rename(columns={
    "teamid": "red_teamid",
}, inplace=True)
red_side.drop(columns=[
    "side",
    "esportsgameid",
    "killerteamid_inhib",
    "killerteamid_tower",
    "killerteamid_kill",
    "gametime_inhib",
    "gametime_tower",
    "gametime_kill",
    "eventtime",
], inplace=True)

joined_side = blue_side.merge(red_side, on="platformgameid")

display(joined_side)
joined_side.to_csv("joined.csv", index=False)

C:\Users\Jan\AppData\Local\Temp\ipykernel_28060\2861731822.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  blue_side.rename(columns={
C:\Users\Jan\AppData\Local\Temp\ipykernel_28060\2861731822.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  blue_side.drop(columns=["side"], inplace=True)
C:\Users\Jan\AppData\Local\Temp\ipykernel_28060\2861731822.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  red_side.rename(columns={
C:\U

,gametime_inhib,platformgameid,killerteamid_inhib,gametime_tower,killerteamid_tower,gametime_kill,killerteamid_kill,eventtime,blue_teamid,esportsgameid,red_teamid
0,1708748,ESPORTSTMNT01:1110148,100,939441,100,245612,100,2019-05-01T09:11:39.980Z,1.007832e+17,101978171958054800,9.956641e+16
1,1602302,ESPORTSTMNT01:1110162,100,677086,100,192537,100,2019-05-01T10:15:29.004Z,9.876799e+16,101978171963953057,9.956641e+16
2,1546286,ESPORTSTMNT01:1110177,100,836608,100,207700,100,2019-05-01T11:15:13.609Z,1.007832e+17,101978171979157453,9.956641e+16
3,1628728,ESPORTSTMNT01:1110187,100,626524,200,359245,200,2019-05-01T12:09:47.993Z,9.876799e+16,101978171965722534,9.956641e+16
4,1694081,ESPORTSTMNT01:1110198,100,895424,100,148483,100,2019-05-01T13:01:30.206Z,9.956641e+16,101978171973586877,9.956641e+16
...,...,...,...,...,...,...,...,...,...,...,...
25182,1700113,LPL_B:330027,100,885600,100,294277,100,2023-07-17T09:14:18.420Z,9.876799e+16,110428848767940944,9.956640e+16
25183,1658796,LPL_B:330032,200,687897,200,227140,100,2023-07-17T10:03:31.166Z,9.956640e+16,110428848767940945,9.876799e+16
25184,1946801,LPL_B:330033,200,900364,200,505054,100,2023-07-17T11:17:17.592Z,9.956640e+16,110428848767875248,9.876799e+16
25185,1447915,LPL_B:330039,200,1008863,100,302191,100,2023-07-17T12:16:05.231Z,9.876799e+16,110428848767875249,9.956640e+16


In [59]:
first_four = pd.read_csv("sql/first_four.csv")
display(first_four)

,eventtime,platformgameid,gametime,winningteam,blue_level,red_level,blue_cs,red_cs
0,2022-03-02T19:15:03.099Z,ESPORTSTMNT01:2734334,210003,200,17,13,65.0,49.0
1,2021-12-22T10:16:45.215Z,ESPORTSTMNT02:2440617,210752,100,15,13,51.0,51.0
2,2019-07-05T11:11:20.733Z,ESPORTSTMNT03:1121320,210723,200,13,15,53.0,61.0
3,2023-02-12T18:16:06.723Z,ESPORTSTMNT01:3312264,210021,100,16,13,64.0,43.0
4,2022-09-04T10:16:32.977Z,ESPORTSTMNT02:2964845,210017,200,12,14,40.0,51.0
...,...,...,...,...,...,...,...,...
25235,2022-08-02T16:00:17.491Z,ESPORTSTMNT03:2885902,210019,200,17,15,69.0,66.0
25236,2020-08-16T12:26:56.052Z,ESPORTSTMNT03:1441062,210891,100,14,15,52.0,54.0
25237,2023-08-06T10:27:43.824Z,LPL_A:343937,210017,100,17,18,61.0,68.0
25238,2023-07-31T20:10:49.662Z,ESPORTSTMNT01:3404526,210025,200,16,17,59.0,55.0
